In [5]:
import json
import os
import tqdm

from tqdm import tqdm

In [9]:
# load the result_comparison sentences with manuscript and cited paper ids
result_comps = []
with open("../data/citation_generation/result_comps.jsonl", "r") as f:
    for line in f:
        p = json.loads(line.strip("\n"))
        for k in set(p.keys()): 
            if k.endswith("prob"): del p[k]
        del p["intent"]
        result_comps.append(p)
        
result_comps[0]

{'citation_sentence': 'Such a discrepancy may likely be due to dose-or species-specific differences since an acute administration of 3.3 mg/kg MDMA did not stimulate locomotion in mice (Scearce-Levie et al, 1999) , but did in rats ( Bankson and Cunningham, 2002) .',
 'manuscript_id': '8281923',
 'cited_id': '6704669'}

In [7]:
# get all involved paper ids
manuscript_ids = {p["manuscript_id"]: [] for p in result_comps}
cited_ids = {p["cited_id"]: [] for p in result_comps}

for i, p in enumerate(result_comps):
    manuscript_ids[p["manuscript_id"]].append(i)
    cited_ids[p["cited_id"]].append(i)

In [11]:
# function to extract result_text from manuscripts
def get_result_text(body_text):
    result_text = []

    i = 0
    is_result = 0
    is_discuss = False 
    while i < len(body_text) and not is_discuss:
        para = body_text[i]
        i += 1
        
        section_name = para["section"].lower()
        is_result += int("result" in section_name or "find" in section_name)
        is_discuss = "discuss" in section_name or "conclu" in section_name

        if is_result and not is_discuss: 
            result_text.append(para["text"])

    return result_text

In [12]:
# for each paper in s2orc,
# append its text to the result_comparison sentences that are involved with it
with open("../data/s2orc/s2orc_result_subset.jsonl", "r") as f:
    for i, line in tqdm(enumerate(f)):
        p = json.loads(line)
        p_id = p["paper_id"]
        
        if p_id in manuscript_ids: 
            result_text = get_result_text(p["body_text"])
            for row_id in manuscript_ids[p_id]:
                result_comps[row_id]["manuscript_text"] = result_text
                
        if p_id in cited_ids: 
            cited_text = []
            for para in p["body_text"]:
                cited_text.append({k: para[k] for k in ("section", "text")})
                
            for row_id in cited_ids[p_id]:
                result_comps[row_id]["cited_text"] = cited_text

982974it [02:02, 8020.90it/s]


In [17]:
with open("../data/citation_generation/dataset.jsonl", "w") as outfile:
    for p in result_comps:
        outfile.write(json.dumps(p) + "\n")